In [1]:
import torch
import torch.nn as nn
import os
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
from torch.optim import Adam
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu" )

In [2]:
def activation(name):
    if name in ['tanh', 'Tanh']:
        return nn.Tanh()
    elif name in ['relu', 'ReLU']:
        return nn.ReLU(inplace=True)
    elif name in ['lrelu', 'LReLU']:
        return nn.LeakyReLU(inplace=True)
    elif name in ['sigmoid', 'Sigmoid']:
        return nn.Sigmoid()
    elif name in ['softplus', 'Softplus']:
        return nn.Softplus(beta=4)
    elif name in ['celu', 'CeLU']:
        return nn.CELU()
    elif name in ['elu']:
        return nn.ELU()
    elif name in ['mish']:
        return nn.Mish()
    else:
        raise ValueError('Unknown activation function')




################################################################
#  1d fourier layer
################################################################
class SpectralConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, modes1):
        super(SpectralConv1d, self).__init__()
        
        self.regularization_exp = 1
        self.regularization_param = 0.0000001

        """
        1D Fourier layer. It does FFT, linear transform, and Inverse FFT.    
        """

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.modes1 = modes1

        self.scale = (1 / (in_channels * out_channels))
        self.weights1 = nn.Parameter(self.scale * torch.rand(in_channels, out_channels, self.modes1, dtype=torch.complex128))

    
    # Complex multiplication
    def compl_mul1d(self, input, weights):
        #print("einsum: input, weights", input.shape, weights.shape)
        # (batch, in_channel, x ), (in_channel, out_channel, x) -> (batch, out_channel, x)

        return torch.einsum("bix,iox->box", input, weights)

    def forward(self, x):
        batchsize = x.shape[0]
        # x.shape == [batch_size, in_channels, number of grid points]
        # hint: use torch.fft library torch.fft.rfft
        # use DFT to approximate the fourier transform
        
        # Compute Fourier coefficients
        #print("x before fft", x.shape)
        x_ft = torch.fft.rfft(x)
        #print("x after fft", x.shape)

        # Multiply relevant Fourier modes
        out_ft = torch.zeros(batchsize, self.out_channels, x.size(-1) // 2 + 1, device=x.device, dtype=torch.complex128)
        #print("outft shape", out_ft.shape)
        out_ft[:, :, :self.modes1] = self.compl_mul1d(x_ft[:, :,:self.modes1], self.weights1)

        # Return to physical space
        x = torch.fft.irfft(out_ft, n=x.size(-1))
        return x
    
    def regularization(self):
        reg_loss = 0
        for name, param in self.named_parameters():
            if 'weight' in name:
                reg_loss = reg_loss + torch.norm(param, self.regularization_exp)
        return self.regularization_param * reg_loss
    


####################################################################


class FNO1d(nn.Module):
    def __init__(self, modes, width):
        super(FNO1d, self).__init__()
        self.modes1 = modes
        self.width = width
        self.padding = 1  # pad the domain if input is non-periodic
        self.linear_p = nn.Linear(3, self.width, dtype=torch.float64)

        self.spect1 = SpectralConv1d(self.width, self.width, self.modes1)
        self.spect2 = SpectralConv1d(self.width, self.width, self.modes1)
        self.spect3 = SpectralConv1d(self.width, self.width, self.modes1)
        self.lin0 = nn.Conv1d(self.width, self.width, 1, dtype = torch.float64)
        self.lin1 = nn.Conv1d(self.width, self.width, 1, dtype = torch.float64)
        self.lin2 = nn.Conv1d(self.width, self.width, 1, dtype = torch.float64)

        self.linear_q = nn.Linear(self.width, 32, dtype = torch.float64)
        self.output_layer = nn.Linear(32, 2, dtype = torch.float64)

        self.activation = torch.nn.Tanh()

    def fourier_layer(self, x, spectral_layer, conv_layer):
        return self.activation(spectral_layer(x) + conv_layer(x))

    def linear_layer(self, x, linear_transformation):
        return self.activation(linear_transformation(x))
    
    def forward(self, x):
        # grid = self.get_grid(x.shape, x.device)
        # x = torch.cat((x, grid), dim=-1)
        #print("x before linear_ü",x.shape)
        x = self.linear_p(x)
        #print("x after linear_p",x.shape)
        x = x.permute(0, 2, 1)
        #print("x after permute",x.shape)

        x = F.pad(x, [0, self.padding])  # pad the domain if input is non-periodic

        x = self.fourier_layer(x, self.spect1, self.lin0)
        x = self.fourier_layer(x, self.spect2, self.lin1)
        x = self.fourier_layer(x, self.spect3, self.lin2)

        x = x[..., :-self.padding]  # pad the domain if input is non-periodic

        x = x.permute(0, 2, 1)
        #print("x after permute",x.shape)

        x = self.linear_layer(x, self.linear_q)
        x = self.output_layer(x)
        return x
    
    def compute_loss(self, pred, train):
        l = torch.nn.MSELoss()
        loss = l(pred, train) + self.spect1.regularization() + self.spect2.regularization() + self.spect3.regularization()
        #loss = tuple(loss)
        return loss
    

    

In [3]:
torch.manual_seed(0)
np.random.seed(0)
import pandas as pd
n_train = 10

df = pd.read_csv(r'C:\Users\matth\OneDrive\Documents\TrainingData.txt')
data_read= df.iloc[:,0:3].values

data_read = torch.tensor(data_read).to(torch.float64)
print(data_read.shape)

Time_Column = torch.clone(data_read[:, 0])
Tf_Column = torch.clone(data_read[:, 1])
Ts_Column = torch.clone(data_read[:, 2])


data_read[:, 0] = Tf_Column
data_read[:, 1] = Ts_Column
data_read[:, 2] = Time_Column


max_Tf = torch.max(data_read[:,0])
max_Ts = torch.max(data_read[:,1])


data_read[:,0] /= max_Tf
data_read[:,1] /= max_Ts
data_read[:,2] /= 602168.58


data_read = data_read.unsqueeze(0)


data_altered = torch.tensor(()).to(torch.float64)

for i in range (0, 12):
    data_altered = torch.cat([data_altered, data_read[:,15*i:45+15*i]], dim=0)

print("data_altered shape", data_altered.shape)
    
x_data = data_altered[0:11,:,0:3]
y_data = data_altered[1:12,:,0:2]

input_function_train = x_data.to(device)#[:n_train, :].to(device)
output_function_train = y_data.to(device)#[:n_train, :].to(device)
#input_function_test = x_data[n_train:, :].to(device)
#output_function_test = y_data[n_train:, :].to(device)
print(input_function_train.shape)
print(output_function_train.shape)
#print(output_function_test.shape)


batch_size = 11 

training_set = DataLoader(TensorDataset(input_function_train, output_function_train), batch_size=batch_size, shuffle=False)
#testing_set = DataLoader(TensorDataset(input_function_test, output_function_test), batch_size=batch_size, shuffle=False)



torch.Size([210, 3])
data_altered shape torch.Size([12, 45, 3])
torch.Size([11, 45, 3])
torch.Size([11, 45, 2])


In [5]:
learning_rate = 0.001

# epochs = 250
epochs = 2500
step_size = 80
gamma = 0.75

modes = 24
width = 256

# model

fno = FNO1d(modes, width).to(device)

optimizer = Adam(fno.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)


l = torch.nn.MSELoss()
freq_print = 1
for epoch in range(epochs):
    train_mse = 0.0
    for step, (input_batch, output_batch) in enumerate(training_set):
        optimizer.zero_grad()
        output_pred_batch = fno(input_batch)
        output_pred_batch = fno(input_batch).squeeze(0)
        output_batch = output_batch.squeeze(0)
        loss_f = torch.log10(l(output_pred_batch, output_batch))
        loss_f.backward()
        optimizer.step()
        train_mse += loss_f.item()
    train_mse /= len(training_set)

    scheduler.step()
    """
    with torch.no_grad():
        fno.eval()
        test_relative_l2 = 0.0
        for step, (input_batch, output_batch) in enumerate(testing_set):
            output_pred_batch = fno(input_batch).squeeze(2)
            loss_f = l(output_pred_batch, output_batch) 
            test_relative_l2 += loss_f.item()
        test_relative_l2 /= len(testing_set)
    """
        
    if epoch % freq_print == 0: print("######### Epoch:", epoch, " ######### Train Loss:", train_mse,)# "######### Relative L2 Test Norm:", test_relative_l2)
    


######### Epoch: 0  ######### Train Loss: -0.11055309070872954
######### Epoch: 1  ######### Train Loss: -1.10886292567902
######### Epoch: 2  ######### Train Loss: -0.9416773119065521
######### Epoch: 3  ######### Train Loss: -0.8537742543829161
######### Epoch: 4  ######### Train Loss: -1.3984525398782899
######### Epoch: 5  ######### Train Loss: -1.3849240144640218
######### Epoch: 6  ######### Train Loss: -1.2297886111041627
######### Epoch: 7  ######### Train Loss: -1.744871105765376
######### Epoch: 8  ######### Train Loss: -1.5532767805776595
######### Epoch: 9  ######### Train Loss: -1.649130533547141
######### Epoch: 10  ######### Train Loss: -1.518766617130403
######### Epoch: 11  ######### Train Loss: -1.6823533034739904
######### Epoch: 12  ######### Train Loss: -2.275048557822493
######### Epoch: 13  ######### Train Loss: -2.27892104650074
######### Epoch: 14  ######### Train Loss: -2.1906397499073718
######### Epoch: 15  ######### Train Loss: -2.3220791685095747
#########

######### Epoch: 131  ######### Train Loss: -4.031734602835857
######### Epoch: 132  ######### Train Loss: -4.269645869633902
######### Epoch: 133  ######### Train Loss: -4.331586969210686
######### Epoch: 134  ######### Train Loss: -3.9991384072989122
######### Epoch: 135  ######### Train Loss: -4.050958879760914
######### Epoch: 136  ######### Train Loss: -4.088640959498062
######### Epoch: 137  ######### Train Loss: -4.108949609291169
######### Epoch: 138  ######### Train Loss: -4.180397886772004
######### Epoch: 139  ######### Train Loss: -4.452303182906191
######### Epoch: 140  ######### Train Loss: -3.806162473152483
######### Epoch: 141  ######### Train Loss: -3.7415381716957894
######### Epoch: 142  ######### Train Loss: -4.0205286294329365
######### Epoch: 143  ######### Train Loss: -4.015027810298599
######### Epoch: 144  ######### Train Loss: -3.9979350112561054
######### Epoch: 145  ######### Train Loss: -4.354112711065145
######### Epoch: 146  ######### Train Loss: -4.2301

######### Epoch: 261  ######### Train Loss: -4.648459546805095
######### Epoch: 262  ######### Train Loss: -4.410007708132302
######### Epoch: 263  ######### Train Loss: -4.591382837670781
######### Epoch: 264  ######### Train Loss: -4.577029946833977
######### Epoch: 265  ######### Train Loss: -4.561080327520365
######### Epoch: 266  ######### Train Loss: -4.87725723713333
######### Epoch: 267  ######### Train Loss: -5.183960753464865
######### Epoch: 268  ######### Train Loss: -4.22201466228376
######### Epoch: 269  ######### Train Loss: -3.8161526583550684
######### Epoch: 270  ######### Train Loss: -3.7145058535072977
######### Epoch: 271  ######### Train Loss: -3.7664862073939136
######### Epoch: 272  ######### Train Loss: -3.972119998807819
######### Epoch: 273  ######### Train Loss: -4.405767943940938
######### Epoch: 274  ######### Train Loss: -4.5073228687617455
######### Epoch: 275  ######### Train Loss: -4.318862084082297
######### Epoch: 276  ######### Train Loss: -4.462459

######### Epoch: 391  ######### Train Loss: -4.822919516787632
######### Epoch: 392  ######### Train Loss: -5.217391011385782
######### Epoch: 393  ######### Train Loss: -5.146947858420121
######### Epoch: 394  ######### Train Loss: -5.21002961234255
######### Epoch: 395  ######### Train Loss: -5.386210888347959
######### Epoch: 396  ######### Train Loss: -4.782789818797527
######### Epoch: 397  ######### Train Loss: -4.404801584513326
######### Epoch: 398  ######### Train Loss: -4.368183649729213
######### Epoch: 399  ######### Train Loss: -4.545611492260603
######### Epoch: 400  ######### Train Loss: -5.012145478135211
######### Epoch: 401  ######### Train Loss: -5.247542390493233
######### Epoch: 402  ######### Train Loss: -5.286786333124763
######### Epoch: 403  ######### Train Loss: -5.0555469119987935
######### Epoch: 404  ######### Train Loss: -5.316490669337481
######### Epoch: 405  ######### Train Loss: -5.198136183926898
######### Epoch: 406  ######### Train Loss: -5.00559074

######### Epoch: 521  ######### Train Loss: -5.417392540591667
######### Epoch: 522  ######### Train Loss: -5.382771156946149
######### Epoch: 523  ######### Train Loss: -5.187350453355789
######### Epoch: 524  ######### Train Loss: -5.597129508512439
######### Epoch: 525  ######### Train Loss: -5.284667653523681
######### Epoch: 526  ######### Train Loss: -5.012697374764043
######### Epoch: 527  ######### Train Loss: -5.1751693990031
######### Epoch: 528  ######### Train Loss: -5.517165756604138
######### Epoch: 529  ######### Train Loss: -5.587250462426965
######### Epoch: 530  ######### Train Loss: -5.491592026213395
######### Epoch: 531  ######### Train Loss: -5.304488543562517
######### Epoch: 532  ######### Train Loss: -5.9525414276139275
######### Epoch: 533  ######### Train Loss: -4.894235453091961
######### Epoch: 534  ######### Train Loss: -4.413061397060264
######### Epoch: 535  ######### Train Loss: -4.201101767029631
######### Epoch: 536  ######### Train Loss: -4.092194638

...

######### Epoch: 1945  ######### Train Loss: -8.395187918545776
######### Epoch: 1946  ######### Train Loss: -8.383800864345442
######### Epoch: 1947  ######### Train Loss: -8.40290944925216
######### Epoch: 1948  ######### Train Loss: -8.441577788557677
######### Epoch: 1949  ######### Train Loss: -8.494674668243752
######### Epoch: 1950  ######### Train Loss: -8.561854766850743
######### Epoch: 1951  ######### Train Loss: -8.645142536936273
######### Epoch: 1952  ######### Train Loss: -8.743850747080039
######### Epoch: 1953  ######### Train Loss: -8.851232687476761
######### Epoch: 1954  ######### Train Loss: -8.96797198841458
######### Epoch: 1955  ######### Train Loss: -9.124522779949118
######### Epoch: 1956  ######### Train Loss: -9.300761135568237
######### Epoch: 1957  ######### Train Loss: -9.34488442199533
######### Epoch: 1958  ######### Train Loss: -9.679967797495607
######### Epoch: 1959  ######### Train Loss: -9.508713261394265
######### Epoch: 1960  ######### Train Loss

######### Epoch: 2074  ######### Train Loss: -8.122713022910398
######### Epoch: 2075  ######### Train Loss: -8.145697707037462
######### Epoch: 2076  ######### Train Loss: -8.187404623256464
######### Epoch: 2077  ######### Train Loss: -8.248706148830342
######### Epoch: 2078  ######### Train Loss: -8.331957415645443
######### Epoch: 2079  ######### Train Loss: -8.441083475639191
######### Epoch: 2080  ######### Train Loss: -8.58106850675176
######### Epoch: 2081  ######### Train Loss: -8.710810782634093
######### Epoch: 2082  ######### Train Loss: -8.859947913969478
######### Epoch: 2083  ######### Train Loss: -9.004913957267382
######### Epoch: 2084  ######### Train Loss: -9.098671258657829
######### Epoch: 2085  ######### Train Loss: -9.129500342449237
######### Epoch: 2086  ######### Train Loss: -9.159399804849475
######### Epoch: 2087  ######### Train Loss: -9.261238313041622
######### Epoch: 2088  ######### Train Loss: -9.515884263654065
######### Epoch: 2089  ######### Train Lo

######### Epoch: 2202  ######### Train Loss: -9.955731239272433
######### Epoch: 2203  ######### Train Loss: -10.066955625267054
######### Epoch: 2204  ######### Train Loss: -10.363351546221475
######### Epoch: 2205  ######### Train Loss: -10.333422295428038
######### Epoch: 2206  ######### Train Loss: -10.80693869122523
######### Epoch: 2207  ######### Train Loss: -10.326843997892068
######### Epoch: 2208  ######### Train Loss: -10.641866129510783
######### Epoch: 2209  ######### Train Loss: -10.092759915126901
######### Epoch: 2210  ######### Train Loss: -9.939934298167792
######### Epoch: 2211  ######### Train Loss: -10.268303132991312
######### Epoch: 2212  ######### Train Loss: -10.145388249958575
######### Epoch: 2213  ######### Train Loss: -9.976231872975275
######### Epoch: 2214  ######### Train Loss: -10.302192341815848
######### Epoch: 2215  ######### Train Loss: -10.377214255814609
######### Epoch: 2216  ######### Train Loss: -10.350570218894203
######### Epoch: 2217  ######

######### Epoch: 2330  ######### Train Loss: -9.436081220109678
######### Epoch: 2331  ######### Train Loss: -9.497202993048822
######### Epoch: 2332  ######### Train Loss: -9.66496020536773
######### Epoch: 2333  ######### Train Loss: -10.020060175802671
######### Epoch: 2334  ######### Train Loss: -10.705534644263969
######### Epoch: 2335  ######### Train Loss: -10.237706616569554
######### Epoch: 2336  ######### Train Loss: -9.909761727153668
######### Epoch: 2337  ######### Train Loss: -9.828447842399454
######### Epoch: 2338  ######### Train Loss: -9.890843215144125
######### Epoch: 2339  ######### Train Loss: -10.102219620405066
######### Epoch: 2340  ######### Train Loss: -10.556391258441982
######### Epoch: 2341  ######### Train Loss: -10.592142264738596
######### Epoch: 2342  ######### Train Loss: -10.245131544178339
######### Epoch: 2343  ######### Train Loss: -10.305366973465825
######### Epoch: 2344  ######### Train Loss: -11.036918440962138
######### Epoch: 2345  #########

######### Epoch: 2458  ######### Train Loss: -9.914586760722681
######### Epoch: 2459  ######### Train Loss: -10.159861134526144
######### Epoch: 2460  ######### Train Loss: -10.63630109088425
######### Epoch: 2461  ######### Train Loss: -10.822278812362983
######### Epoch: 2462  ######### Train Loss: -10.762731181839587
######### Epoch: 2463  ######### Train Loss: -10.812183135327166
######### Epoch: 2464  ######### Train Loss: -10.604475348084826
######### Epoch: 2465  ######### Train Loss: -10.640416856661952
######### Epoch: 2466  ######### Train Loss: -10.854504405455238
######### Epoch: 2467  ######### Train Loss: -10.869857720778942
######### Epoch: 2468  ######### Train Loss: -11.082731640343447
######### Epoch: 2469  ######### Train Loss: -10.967318099846832
######### Epoch: 2470  ######### Train Loss: -11.254565741920876
######### Epoch: 2471  ######### Train Loss: -10.832616627283404
######### Epoch: 2472  ######### Train Loss: -10.832690155926311
######### Epoch: 2473  ####

In [1]:
test_read = pd.read_csv(r'C:\Users\matth\OneDrive\Documents\TestingData.txt')
test_read = test_read.iloc[:,0:1].values
test_read = torch.tensor(test_read).to(torch.float64).squeeze(1).to(device)
test_read /= 602168.58
print("input_function_train[0,:,:] shape", input_function_train[0,:,:].shape)
print(602168.58*input_function_train[0,44,2],602168.58*input_function_train[3,0,2] )
output_function_test_0 = fno(input_function_train[0,:,:].unsqueeze(0))
output_function_test_1 = fno(input_function_train[3,:,:].unsqueeze(0))
output_function_test_2 = fno(input_function_train[6,:,:].unsqueeze(0))
output_function_test_3 = fno(input_function_train[9,:,:].unsqueeze(0))
next_step = torch.cat((output_function_train[10,:,:].unsqueeze(0), data_read[:,150:195, 2].unsqueeze(2).to(device)), dim=2)
output_function_test_4 = fno(data_read[:,150:195,:].to(device))
#output_function_test_5 = fno(data_read[:,165:210].to(device))
interval = torch.linspace(446050.8,669076.2, 90).to(torch.float64)/602168.58

input_function_pred = torch.cat((input_function_train[3,:,2], input_function_train[6,:,2], input_function_train[9,:,2], interval.to(device)), dim =0)
output_function_pred = torch.cat((output_function_test_0, output_function_test_1, output_function_test_2, output_function_test_3, output_function_test_4), dim=1)


input_function_plot = torch.cat((input_function_train[0,:,2], input_function_train[3,:,2], input_function_train[6,:,2], input_function_train[9,:,2], ), dim=0)
output_function_plot = torch.cat((input_function_train[0,:,0], input_function_train[3,:,0], input_function_train[6,:,0], input_function_train[9,:,0],), dim=0)

#loss = torch.nn.MSELoss()
#print("loss is",torch.log10(loss(output_function_train[0,:,:], output_function_test_0.squeeze(0))))


plt.figure(dpi=250)
plt.grid(True, which="both", ls=":")

plt.plot(input_function_plot.cpu().detach(), output_function_plot.cpu().detach(), label="True Solution", c="C0", lw=2)
plt.plot(input_function_pred.cpu().detach(), output_function_pred[:,:,0].squeeze(0).cpu().detach(), label="Approximate Solution", c="C1", lw=2)

plt.legend()
plt.show()


NameError: name 'pd' is not defined

In [ ]:
test_read = pd.read_csv(r'C:\Users\matth\OneDrive\Documents\TestingData.txt')
test_read = test_read.iloc[:,0:1].values
test_read = torch.tensor(test_read).to(torch.float64).squeeze(1).to(device)
test_read /= 602168.58

saved_pred = pd.read_csv(r'C:\Users\matth\OneDrive\Documents\tensor_data.txt')
saved_pred = saved_pred.iloc[:,1:3].values
saved_pred = torch.tensor(saved_pred).to(torch.float64).unsqueeze(0).to(device)


output_function_test_0 = fno(input_function_train[0,:,:].unsqueeze(0))
output_function_test_1 = fno(input_function_train[1,:,:].unsqueeze(0))
output_function_test_2 = fno(input_function_train[2,:,:].unsqueeze(0))
output_function_test_3 = fno(input_function_train[3,:,:].unsqueeze(0))
output_function_test_4 = fno(input_function_train[4,:,:].unsqueeze(0))
output_function_test_5 = fno(data_read[:,175:210].to(device))

saved_pred[:,:,0] /= max_Tf
saved_pred[:,:,1] /= max_Ts
print(saved_pred.shape)
input_function_pred = 602168.58 * torch.cat((input_function_train[1,:,2], input_function_train[2,:,2], input_function_train[3,:,2], input_function_train[4,:,2], data_read[:,175:210,2].squeeze(0).to(device), test_read), dim=0)
output_function_pred = torch.cat((output_function_test_0, output_function_test_1, output_function_test_2, output_function_test_3, output_function_test_4, saved_pred), dim=1)
print(input_function_pred.shape, output_function_pred.shape)

input_function_plot = 602168.58 * torch.cat((input_function_train[0,:,2], input_function_train[1,:,2], input_function_train[2,:,2], input_function_train[3,:,2], input_function_train[4,:,2], data_read[:,175:210,2].squeeze(0).to(device)), dim=0)
output_function_plot = torch.cat((input_function_train[0,:,0:2], input_function_train[1,:,0:2], input_function_train[2,:,0:2], input_function_train[3,:,0:2], input_function_train[4,:,0:2], output_function_train[4,:,0:2]), dim=0)



plt.figure(dpi=1000)
plt.grid(True, which="both", ls=":")
print(output_function_pred.shape)
plt.plot(input_function_pred.cpu().detach(), max_Tf * output_function_pred[:,:,0].squeeze(0).cpu().detach(), label="Approximate Solution", c="C1", lw=0.5)
plt.plot(input_function_plot.cpu().detach(), max_Tf * output_function_plot[:,0].cpu().detach(), label="True Solution", c="C0", lw=0.5)


plt.legend()
plt.show()

plt.figure(dpi=250)
plt.grid(True, which="both", ls=":")

plt.plot(input_function_pred.cpu().detach(), max_Ts * output_function_pred[:,:,1].squeeze(0).cpu().detach(), label="Approximate Solution", c="C1", lw=2)
plt.plot(input_function_plot.cpu().detach(), max_Ts * output_function_plot[:,1].cpu().detach(), label="True Solution", c="C0", lw=2)


plt.legend()
plt.show()


In [ ]:
# Create a DataFrame from the numpy array
dt = pd.read_csv(r'C:\Users\matth\OneDrive\Documents\TestingData.txt')
test_read = dt.iloc[:,0:1].values
test_read = torch.tensor(test_read).to(torch.float64).squeeze(1).to(device)



print("input shapes:", data_read[:,165:210,:].shape)
output_function_test_0 = fno(data_read[:,165:210,:].to(device))
print("output_function_test_0",output_function_test_0.shape)



next_step_temp = torch.cat((data_read[:,180:210,0:2], output_function_test_0[:,30:,:]), dim=1)
next_step_time = torch.linspace(446050.8,557563.5, 45, dtype = torch.float64)/602168.58
next_step_time = next_step_time.unsqueeze(0).unsqueeze(2)
next_step = torch.cat((next_step_temp, next_step_time), dim=2)
print("next_step",next_step.shape)
output_function_test_1 = fno(next_step)


next_step_temp = torch.cat((data_read[:,195:210,0:2],output_function_test_0[:,30:,:] , output_function_test_1[:,30:,:]), dim=1)
next_step_time = torch.linspace(483221.7, 594734.4, 45, dtype = torch.float64)/602168.58
next_step_time = next_step_time.unsqueeze(0).unsqueeze(2)
next_step = torch.cat((next_step_temp, next_step_time), dim=2)
print("next_step",next_step.shape)
output_function_test_2 = fno(next_step)

print("output shapes:",output_function_test_1.shape, output_function_test_2.shape)
write_output = torch.cat((output_function_test_0[:,30:,:], output_function_test_1[:,30:,:], output_function_test_2[:,30:34,:]), dim=1)
print("write_output", write_output.shape)


submission = torch.empty(34,3)
submission[:, 0] = test_read
submission[:, 1:3] = write_output[0,:34, 0:2] 
submission[:, 1] *= max_Tf
submission[:, 2] *= max_Ts


submission = pd.DataFrame(submission.detach().numpy())

# Save the DataFrame as a text file
submission.to_csv(r'C:\Users\matth\OneDrive\Documents\submission3_morenodes.txt', sep=',', index=False, header=['t', 'tf0', 'ts0'])

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
data_tens = pd.read_csv(r'C:\Users\matth\OneDrive\Documents\tensor_data.txt')
print(data_tens.shape)
subp = pd.read_csv(r'C:\Users\matth\Downloads\submissiontask3.txt')
y_1 = data_tens.iloc[:,0:3].values
y_2 = subp.iloc[:,0:3].values

y_1 = torch.tensor(y_1).to(torch.float64).squeeze(1)
y_2 = torch.tensor(y_2).to(torch.float64).squeeze(1)



plt.figure(dpi=250)
plt.grid(True, which="both", ls=":")

plt.plot(y_1[:,0].cpu().detach(), y_1[:,1].cpu().detach(), label="Matthias Solution", c="C1", lw=1)
plt.plot(y_2[:,0].cpu().detach(), y_2[:,1].cpu().detach(), label="Paul Solution", c="C0", lw=1)
plt.legend()
plt.show()
